In [ ]:
from pdf_scraper.doc_utils   import (open_exam, get_doc_line_df, identify_section_headers, 
                                     identify_text_headers,identify_footers, identify_instructions, 
                                     identify_subtitles, identify_subsubtitles,get_images,preproc_images, 
                                     assign_in_image_captions, identify_vertical_captions,
                                     identify_all_page_clusters, enrich_doc_df_with_images)

from pdf_scraper.line_utils    import clean_line_df, get_category_boxes, get_df_bbox
from pdf_scraper.image_utils   import get_bboxed_page_image,show_image, show_all_imgs
from pdf_scraper.general_utils import bbox_vert_dist, bbox_horiz_dist
from time import sleep
import numpy as np 
import pandas as pd

import os, io, re

from IPython.display import display, clear_output, Image
import matplotlib.pyplot as plt

In [ ]:
def get_parsed_df(doc):
    df = get_doc_line_df(doc)
    doc_width = doc[0].rect.width

    images = preproc_images(get_images(doc))
    assign_in_image_captions(df, images)

    df = clean_line_df(df)
    df = enrich_doc_df_with_images(df, images)

    identify_all_page_clusters(df, 2.0/3.0, 1.15, text_only=True)
    identify_footers(df)
    identify_instructions(df)
    identify_section_headers(df)
    identify_text_headers(df, doc_width)
    identify_subtitles(df, doc_width)
    identify_subsubtitles(df, doc_width)

    return df

In [ ]:
def get_caption_input(df, page):
    while True:
       caption_input = input("Caption text: ").strip().lower()
       if caption_input == "":
           break

       matches = df[(df.page == page) & (df.text.str.strip().str.lower().str.contains(caption_input)) ]
       if matches.empty:
           print("No match found. Retype, perhaps with more of line.")
           continue
       elif len(matches) > 1: 
           print(f"{len(matches)} matches:")
           display(matches.head(10))
           index_input = input("Select index: ")
           index_input = int(index_input)
           matches     = df.loc[[index_input]]

       df.loc[matches.index, "category"] = "caption2"
       print(f"Marked: {matches.text.values[0]} as caption2.")

In [ ]:
def update_image(image_handle, img):
    buf = io.BytesIO()
    plt.figure(figsize=(16, 10))
    plt.imshow(img); plt.axis("off"); plt.tight_layout()
    plt.savefig(buf, format="png")
    plt.close()
    buf.seek(0)

    image_handle.update(Image(data=buf.read()))
    return image_handle
    

In [ ]:
dfs = []
output_csv = "captioned_dfs.csv"

for year in range(2020, 2021):
    clear_output(wait=True)
    print(f"\n----- Year: {year} -----\n")
    doc         = open_exam(year)
    images      = preproc_images(get_images(doc))
    df          = get_parsed_df(doc)
    df["year"]  = year

    image_handle = display(None, display_id=True)
    if year in [y for y in range(2001, 2011)]:
        file_exists = os.path.exists(output_csv)
        df.to_csv(output_csv, mode="a", header=not file_exists, index=False)
        dfs.append(df)
        continue
        
    image_pages = set(img["page"] for img in images if img["page"]>1)
    for page in sorted(image_pages):
        page_df = df[df.page == page]
        rects   = get_category_boxes(page_df, "cluster")
        img     = get_bboxed_page_image(doc, page, rects, labels = np.unique(page_df.cluster))
        
        image_handle = update_image(image_handle, img)
        print(f"\n{year} --- Page {page} ---")
        
        has_captions = input("Are there captions on this page? (y/n): ").strip().lower()
        if has_captions not in ("y", "yes"):
            print("Skipping this page.\n")
            continue
        get_caption_input(df, page)

        display(df.loc[(df.category=="caption2") & (df.page==page),["text","page"] ].head())
        good_page = input("Are these captions correct? (y/n): ").strip().lower()
        if good_page not in ("y", "yes"):
            df.loc[(df.page==page) & (df.category=="caption2"), "category"] = "uncategorised"
            get_caption_input(df, page)
        print(f"Done with page {page}.\n")
    
    file_exists = os.path.exists(output_csv)
    df.to_csv(output_csv, mode="a", header=not file_exists, index=False)

    dfs.append(df)

big_df = pd.concat(dfs, ignore_index=True)
print("\nFinal dataframe shape:", big_df.shape)

# Postprocessing

## Check and clean dataframe

In [ ]:
df = pd.read_csv("captioned_dfs.csv")
df.drop_duplicates(inplace=True)
df.sort_values(by=["year","page"],ignore_index=True,inplace=True)
df.loc[df.category=="caption2",["text","page","year"]]

The below check will identify cases where two labelling run throughs were made and different categorisations
were assigned to the same lines in different run throughs. The code below will allow you to identify and then
remove the incorrectly categorised lines.

In [ ]:
capt_df = df.loc[ df.category=="caption2", ["text","category","year","page"]].copy()
check = df[["text","category","year","page"]].merge(capt_df, "inner", on=["text","year","page"])
assert len(check[check.category_x != check.category_y]) == 0

In [ ]:
# Once you have identified incorrectly categorised lines using the code above, you can drop them using their
# indices as below.
# df.drop([12453,12454],inplace=True)
# df.sort_values(by=["year","page"],ignore_index=True,inplace=True)
# df.to_csv("captioned_dfs.csv")

## Enrich for caption detection

### Drop pages without images

In [ ]:
for year in range(2001, 2026):
    print(year, end=", ")
    doc         = open_exam(year)
    images      = preproc_images(get_images(doc))
    image_pages = set(img["page"] for img in images if img["page"]>1)
    drop_mask   = (df.year == year) & (~df.page.isin(image_pages)) 
    to_drop     = df[drop_mask].index
    df.drop(to_drop, inplace=True)

### Drop unwanted columns

In [ ]:
df = df[["x0","y0","x1","y1", "mode_font","w", "h", "text","font_size","category","page","cluster","year"]]
df.head()

### Drop already categorised lines

In [ ]:
mask = df.category.isin( ("uncategorised","caption2","image") ) 
print(len(df))
df = df[mask]
print(len(df))

### Is line within image bound?

In [ ]:
df.loc[df.category=="caption2", ["text","category"]]

### Convert line_df to block_df

In [ ]:
def text_func(text_series):
    """
    Returns True if any text entry in the series likely refers to an image or figure caption.
    Matches words and patterns like:
      - 'image', 'image 1', 'Image:', etc.
      - 'figure', 'figure 1', 'fig.', 'fig 2', etc.
      - 'above', 'below', with or without colons or punctuation.
    """
    pattern = re.compile(
        r'\b('
        r'image\s*\d*[\s:.,;)]*|'      # image, image 1:, image2, etc.
        r'fig(?:ure)?\s*\d*[\s:.,;)]*|' # fig, fig. 1, figure 2:, etc.
        r'above[\s:.,;)]*|'             # above, above:
        r'below[\s:.,;)]*'              # below, below:
        r')',
        re.IGNORECASE
    )
    return text_series.astype(str).apply(lambda t: bool(pattern.search(t))).any()

agg_dict = {
    "x0": "min",
    "y0": "min",
    "x1": "max",
    "y1": "max",
    "mode_font": lambda x: x.mode().iat[0] if not x.mode().empty else None,
    "w": "median",
    "h": "median",
    "text": lambda x: "\n".join(x.astype(str)),
    #"text": text_func,
    "font_size": "median",
    "category": lambda x: x.mode().iat[0] if not x.mode().empty else None,
}

block_df =df.groupby(["year","page","cluster"]).agg(agg_dict).reset_index()

In [ ]:
block_df.loc[(block_df.category=="image"), "h"]  = block_df[block_df.category=="image"].y1 - block_df.loc[block_df.category=="image"].y0
block_df.loc[(block_df.category=="image"), "w"]  = block_df[block_df.category=="image"].x1 - block_df.loc[block_df.category=="image"].x0
block_df[block_df.category=="image"].head(4)

In [ ]:
capt_block_df = block_df[block_df.category=="caption2"].copy()
print(f'Data is {100*len(capt_block_df)/len(block_df[block_df.category !="image"]):0.2f}% caption.')
capt_block_df

### Find nearest image to block

In [ ]:
# For each block, check the distance to every image on the same page and year, 
# choose block which is closest. dx will have to be 0 for it to be a vertical caption.
# - filter only blocks with dx == 0
# - amongst those return the lowest dy.

In [ ]:
def images_overlapping_with_bbox(bbox, page, images):
    """
    Note: These images must be from the same year as the bbox.
    """
    imgs = [img for img in images if img["page"]==page]
    ov_ims = []
    for img in imgs:
        dx = bbox_horiz_dist(bbox, img["bbox"])
        if dx == 0:
            print(f"overlaps with image: {img["number"]}")
            ov_ims.append(img)
    return ov_ims
        

In [ ]:
img_df

In [ ]:
row = block_df.loc[227]
year   = row.year
page   = row.page
bbox   = row.x0, row.y0, row.x1, row.y1
img_df = block_df[(block_df.category=="image") & (block_df.year==year) & (block_df.page==page)].copy()

img_df["dx"] =img_df.apply( lambda x: bbox_horiz_dist(bbox, (x.x0, x.y0, x.x1, x.y1) ), axis=1)
img_df[img_df.dx==0].index

In [ ]:
def get_overlapping_images(row, block_df):
    """
    Note: These images must be from the same year as the bbox.
    """
    year   = row.year
    page   = row.page
    bbox   = row.x0, row.y0, row.x1, row.y1
    img_df = block_df[(block_df.category=="image") & (block_df.year==year) & (block_df.page==page)].copy()

    img_df["dx"] = img_df.apply( lambda x: bbox_horiz_dist(bbox, (x.x0, x.y0, x.x1, x.y1) ) , axis=1)
    indices = img_df[img_df.dx==0].index
    return indices

In [ ]:
row = block_df.loc[227]

indices = images_overlapping_with_bbox(row, block_df)
img_df = block_df.loc[indices]


bbox1 = row.x0, row.y0, row.x1, row.y1
dy = img_df.apply( lambda x: bbox_vert_dist(bbox1, (x.x0,x.y0,x.x1,x.y1) ), axis=1 )   

dy_min     = dy.min()
nearest_im = img_df.loc[dy.idxmin()]


In [ ]:
def get_nearest_image(row, img_df):
    bbox1 = row.x0, row.y0, row.x1, row.y1
    dy = img_df.apply( lambda x: bbox_vert_dist(bbox1, (x.x0,x.y0,x.x1,x.y1) ), axis=1 )   
    
    nearest_im = img_df.loc[dy.idxmin()]
    return nearest_im

def get_nearest_image_width(row, img_df):
    im =get_nearest_image(row,img_df)
    return im.w

def get_nearest_image_height(row, img_df):
    im =get_nearest_image(row,img_df)
    return im.h

def get_nearest_image_centre(row, img_df):
    im =get_nearest_image(row,img_df)
    return (im.x0 + im.x1)/2

def get_nearest_image_width(row, img_df):
    im =get_nearest_image(row,img_df)
    return im.h

def get_nearest_image_distance(row , img_df):
    bbox1 = row.x0, row.y0, row.x1, row.y1
    dy = img_df.apply( lambda x: bbox_vert_dist(bbox1, (x.x0,x.y0,x.x1,x.y1) ), axis=1 )   
    
    return  dy.min()

In [ ]:
year= 2011
doc = open_exam(year)
images = preproc_images(get_images(doc))
capt_bbox = get_df_bbox( capt_block_df.loc[[177]])
page = capt_block_df.loc[177].page
imgs = images_overlapping_with_bbox(capt_bbox, page, images)

show_all_imgs(1,3, imgs )


In [ ]:
def nearest_overlapping_image(bbox, imgs):
    """
    Of those images which overlap in the x direction, which is the nearest vertically?
    """
    dy_min = 1000.0
    nearest_image = None
    for im in imgs:
        im_bbox = im["bbox"]
        dy = bbox_vert_dist(bbox,im_bbox)
        if  dy < dy_min:
            nearest_image = im
            dy_min = dy
    return (nearest_image, dy_min)

In [ ]:
year= 2011
doc = open_exam(year)
images = preproc_images(get_images(doc))
capt_bbox = get_df_bbox( capt_block_df.loc[[177]])
page = capt_block_df.loc[177].page
imgs = images_overlapping_with_bbox(capt_bbox, page, images)
nearest_img, dy = nearest_overlapping_image(capt_bbox, imgs)

#show_image(img )

In [ ]:
row = capt_block_df.loc[177]
row.year

In [ ]:
def dy_to_nearest_image(row):
    year = row.year

### Is block aligned in x with image